# Demyst - Scientific Integrity Linter

**Demyst** ensures your scientific code means what you think it means. This notebook walks through the five types of issues Demyst can detect.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/Hmbown/demyst/blob/main/notebooks/demo.ipynb)

In [ ]:
# Install Demyst
!pip install demyst -q

## 1. Computational Mirages

Operations that destroy variance information - the signature issue Demyst catches.

**The Problem:** Aggregation functions like `mean()` can hide critical outliers.

In [ ]:
# Example: AI Safety - A Swarm with a Hidden Rogue Agent
mirage_code = '''
import numpy as np

def analyze_swarm_safety(agent_count=1000):
    """Analyze alignment scores of an AI agent swarm."""
    # 999 agents are perfectly aligned (1.0)
    # 1 agent is rogue (0.0) - will cause cascade failure!
    swarm_alignment = np.ones(agent_count)
    swarm_alignment[-1] = 0.0
    
    # The Mirage: mean looks safe but hides the rogue agent
    mean_alignment = np.mean(swarm_alignment)  # Returns 0.999!
    
    if mean_alignment > 0.99:
        print("DEPLOY SWARM")  # Catastrophic decision
'''

# Write to temp file and analyze
with open('/tmp/mirage_example.py', 'w') as f:
    f.write(mirage_code)

!demyst mirage /tmp/mirage_example.py

**Demyst catches this!** The `np.mean()` destroys variance information, hiding the rogue agent.

---

## 2. Data Leakage

The #1 error in machine learning - information from the test set leaks into training.

In [ ]:
# Example: Fitting scaler before train/test split
leakage_code = '''
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import numpy as np

X = np.random.randn(1000, 10)
y = np.random.randint(0, 2, 1000)

# WRONG: Fitting on ALL data before split leaks test statistics!
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)  # Leakage here!
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y)
'''

with open('/tmp/leakage_example.py', 'w') as f:
    f.write(leakage_code)

!demyst leakage /tmp/leakage_example.py

---

## 3. P-Hacking & Statistical Validity

Multiple hypothesis tests without correction inflate false positive rates.

In [ ]:
# Example: Running multiple t-tests without correction
phacking_code = '''
from scipy import stats
import numpy as np

def find_significant_genes(gene_expressions, conditions):
    significant = []
    for gene in gene_expressions:
        control = gene_expressions[gene]["control"]
        treatment = gene_expressions[gene]["treatment"]
        
        _, p_value = stats.ttest_ind(control, treatment)
        
        # PROBLEM: No multiple testing correction!
        # With 1000 genes at alpha=0.05, expect ~50 false positives
        if p_value < 0.05:
            significant.append(gene)
    return significant
'''

with open('/tmp/phacking_example.py', 'w') as f:
    f.write(phacking_code)

!demyst hypothesis /tmp/phacking_example.py

---

## 4. Unit/Dimensional Consistency

Mixing incompatible physical units leads to silent calculation errors.

In [ ]:
# Example: Adding quantities with incompatible dimensions
units_code = '''
def calculate_trajectory(velocity_mps, time_seconds, distance_km):
    """Calculate something with mixed units."""
    # WRONG: velocity (m/s) * time (s) = meters, but distance is in km!
    result = velocity_mps * time_seconds + distance_km
    return result
'''

with open('/tmp/units_example.py', 'w') as f:
    f.write(units_code)

!demyst units /tmp/units_example.py

---

## 5. Deep Learning Integrity

Common tensor and gradient issues in neural networks.

In [ ]:
# Example: Potential gradient death in deep networks
tensor_code = '''
import torch
import torch.nn as nn

class DeepNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        # Very deep network with potential gradient issues
        self.layers = nn.Sequential(
            *[nn.Linear(100, 100) for _ in range(50)]
        )
        # No batch norm, residual connections, or careful initialization
    
    def forward(self, x):
        return self.layers(x)
'''

with open('/tmp/tensor_example.py', 'w') as f:
    f.write(tensor_code)

!demyst tensor /tmp/tensor_example.py

---

## Run All Checks at Once

Use `demyst analyze` to run all detectors on a file or directory.

In [ ]:
# Analyze all examples
!demyst analyze /tmp/mirage_example.py

---

## Programmatic API

You can also use Demyst programmatically in your code:

In [ ]:
from demyst import LeakageHunter

# Analyze code for data leakage
hunter = LeakageHunter()
result = hunter.analyze(leakage_code)

print(f"Issues found: {len(result.issues)}")
for issue in result.issues:
    print(f"  - Line {issue.line}: {issue.message}")

---

## Try It Yourself!

Paste your own code below and run Demyst on it:

In [ ]:
# YOUR CODE HERE - modify this cell!
your_code = '''
import numpy as np

# Try adding some suspicious code here:
# - np.mean() on important data
# - fit_transform() before train_test_split()
# - p < 0.05 checks in loops

data = np.random.randn(100)
result = np.mean(data)  # Will Demyst catch this?
'''

with open('/tmp/your_code.py', 'w') as f:
    f.write(your_code)

!demyst analyze /tmp/your_code.py

---

## Learn More

- **GitHub:** [github.com/Hmbown/demyst](https://github.com/Hmbown/demyst)
- **Documentation:** [demyst.readthedocs.io](https://demyst.readthedocs.io) (coming soon)
- **Install:** `pip install demyst`

**Good Code is Good Science.**